In [1]:
import sys
sys.path.insert(0, '../attention-is-all-you-need/')
sys.path.insert(0, '../modeling/')
sys.argv = ['']

In [41]:
import os
import dill
import random
import glob
import setting
import torch
import numpy as np
import sentencepiece as spm
import argparse
import torch.distributed as dist
#import torch.multiprocessing as mp
import logging
import logging.config

from visdom import Visdom
from utils import fix_torch_randomness, get_sentencepiece, get_number_of_params
from transformer import make_model, subsequent_mask
from dataset import load_dataset_aihub, KRENDataset, KRENField, MyIterator, batch_size_fn
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.autograd import Variable
from setting import *
from feature import *

ImportError: cannot import name 'get_number_of_params'

In [3]:
# define logger
logging.config.fileConfig('../attention-is-all-you-need/logging.conf')
log = logging.getLogger('LaH')

In [4]:
# define ENV variables
os.environ['MASTER_ADDR'] = '127.0.0.1'
os.environ['MASTER_PORT'] = '10002'

In [5]:
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type_as(src.data)
    for i in range(max_len-1):
        out = model.decode(memory, src_mask,
                           Variable(ys),
                           Variable(subsequent_mask(ys.size(1))
                                    .type_as(src.data)))
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim = 1)
        next_word = next_word.item()
        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=1)
    return ys

In [6]:
parser = argparse.ArgumentParser()
parser.add_argument('--world_size', default=setting.world_size, type=int)
parser.add_argument('--multi_gpu', default=setting.multi_gpu, type=bool)
parser.add_argument('--epochs', default=45, type=int)

_StoreAction(option_strings=['--epochs'], dest='epochs', nargs=None, const=None, default=45, type=<class 'int'>, choices=None, help=None, metavar=None)

In [7]:
args = parser.parse_args()

In [8]:
#sent_pairs = load_dataset_aihub(path='../attention-is-all-you-need/data/')
sent_pairs = load_dataset_aihub()

In [12]:
a = 1

In [9]:
src_spm = spm.SentencePieceProcessor()
trg_spm = spm.SentencePieceProcessor()
src_spm.Load('../attention-is-all-you-need/spm/{}.model'.format(src_prefix))
trg_spm.Load('../attention-is-all-you-need/spm/{}.model'.format(trg_prefix))

True

In [10]:
inp_lang, out_lang = src_spm, trg_spm

In [11]:
extra_options = 'bos:eos' #'reverse:bos:eos'
#src_spm.SetEncodeExtraOptions(extra_options)
trg_spm.SetEncodeExtraOptions(extra_options)
log.info('loaded input sentencepiece model: {}'.format(src_prefix))
log.info('loaded output sentencepiece model: {}'.format(trg_prefix))

[INFO] (<ipython-input-11-14a64b6c78f5>:4) > loaded input sentencepiece model: spm-src-50000
[INFO] (<ipython-input-11-14a64b6c78f5>:5) > loaded output sentencepiece model: spm-trg-32000


In [13]:
# split train/valid sentence pairs
n_train = int(len(sent_pairs) * 0.8)
valid_sent_pairs = sent_pairs[n_train:]
log.info('valid_sent_pairs: {}'.format(len(valid_sent_pairs)))

[INFO] (<ipython-input-13-47613aab77f2>:4) > valid_sent_pairs: 320484


In [14]:
# these are used for defining tokenize method and some reserved words
SRC = KRENField(pad_token='<pad>')
TRG = KRENField(pad_token='<pad>')

In [15]:
src_vocab = torch.load('../attention-is-all-you-need/spm/{}.spm'.format(src_prefix), pickle_module=dill)
trg_vocab = torch.load('../attention-is-all-you-need/spm/{}.spm'.format(trg_prefix), pickle_module=dill)
SRC.vocab = src_vocab
TRG.vocab = trg_vocab
log.info('input vocab was loaded: spm/{}.spm'.format(src_prefix))
log.info('output vocab was loaded: spm/{}.spm'.format(trg_prefix))

[INFO] (<ipython-input-15-a799ae4d4d42>:5) > input vocab was loaded: spm/spm-src-50000.spm
[INFO] (<ipython-input-15-a799ae4d4d42>:6) > output vocab was loaded: spm/spm-trg-32000.spm


In [16]:
# make dataloader from KRENDataset
train, valid, test = KRENDataset.splits(sent_pairs, (SRC, TRG), inp_lang, out_lang, encoding_type='pieces')
valid_iter = MyIterator(valid, batch_size=100, device=0,
                        repeat=False, sort_key=lambda x: (len(x.src), len(x.trg)),
                        batch_size_fn=batch_size_fn, train=False)

Encoding as pieces: 100%|██████████| 32644/32644 [00:02<00:00, 16143.98it/s]


In [17]:
# fix torch randomness
fix_torch_randomness()

In [18]:
# define input/output size
args.inp_n_words = src_vocab_size
args.out_n_words = trg_vocab_size
log.info('inp_n_words: {} out_n_words: {}'.format(args.inp_n_words, args.out_n_words))

[INFO] (<ipython-input-18-fe544bfaea37>:4) > inp_n_words: 50000 out_n_words: 32000


In [19]:
# define model
model = make_model(args.inp_n_words, args.out_n_words)

../modeling/transformer.py:27: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(p)


In [20]:
model_name_full_path = '../attention-is-all-you-need/models/model-tmp.bin'
checkpoint = torch.load(model_name_full_path)
state_dict = checkpoint['state_dict']
model.load_state_dict(state_dict)
model.cuda()

EncoderDecoder(
  (encoder): Encoder(
    (layers): ModuleList(
      (0): EncoderLayer(
        (self_attn): MultiHeadedAttention(
          (linears): ModuleList(
            (0): Linear(in_features=512, out_features=512, bias=True)
            (1): Linear(in_features=512, out_features=512, bias=True)
            (2): Linear(in_features=512, out_features=512, bias=True)
            (3): Linear(in_features=512, out_features=512, bias=True)
          )
          (dropout): Dropout(p=0.1)
        )
        (feed_forward): PositionwiseFeedForward(
          (w_1): Linear(in_features=512, out_features=2048, bias=True)
          (w_2): Linear(in_features=2048, out_features=512, bias=True)
          (dropout): Dropout(p=0.1)
        )
        (sublayer): ModuleList(
          (0): SublayerConnection(
            (norm): LayerNorm()
            (dropout): Dropout(p=0.1)
          )
          (1): SublayerConnection(
            (norm): LayerNorm()
            (dropout): Dropout(p=0.1)
      

In [25]:
model.eval()

EncoderDecoder(
  (encoder): Encoder(
    (layers): ModuleList(
      (0): EncoderLayer(
        (self_attn): MultiHeadedAttention(
          (linears): ModuleList(
            (0): Linear(in_features=512, out_features=512, bias=True)
            (1): Linear(in_features=512, out_features=512, bias=True)
            (2): Linear(in_features=512, out_features=512, bias=True)
            (3): Linear(in_features=512, out_features=512, bias=True)
          )
          (dropout): Dropout(p=0.1)
        )
        (feed_forward): PositionwiseFeedForward(
          (w_1): Linear(in_features=512, out_features=2048, bias=True)
          (w_2): Linear(in_features=2048, out_features=512, bias=True)
          (dropout): Dropout(p=0.1)
        )
        (sublayer): ModuleList(
          (0): SublayerConnection(
            (norm): LayerNorm()
            (dropout): Dropout(p=0.1)
          )
          (1): SublayerConnection(
            (norm): LayerNorm()
            (dropout): Dropout(p=0.1)
      

In [27]:
for i, batch in enumerate(valid_iter):
    if i == 20:
        break
    src = batch.src.transpose(0, 1)[:1]
    src_mask = (src != SRC.vocab.stoi["<pad>"]).unsqueeze(-2)

    
    out = greedy_decode(model, src.cuda(), src_mask.cuda(),
                        max_len=60, start_symbol=TRG.vocab.stoi["<s>"])
    
    
    print("Input::", end="\t")
    for i in range(src.size(1)):
        sym = SRC.vocab.itos[src[0, i]]
        if sym == "</s>": break
        print(sym, end =" ")
    print('')
    
    print("Translation:", end="\t")
    for i in range(1, out.size(1)):
        sym = TRG.vocab.itos[out[0, i]]
        if sym == "</s>": break
        print(sym, end =" ")
    print('')
    print("Target:", end="\t")
    for i in range(1, batch.trg.size(0)):
        sym = TRG.vocab.itos[batch.trg.data[i, 0]]
        if sym == "</s>": break
        print(sym, end =" ")
    print('')
    print('---------------')

Input::	▁집으로 <unk> 나요 ? 
Translation:	▁Doesn ' t ▁it ▁live ▁in ▁the ▁house ? 
Target:	▁Will ▁it ▁be ▁delivered ▁to ▁my ▁home ? 
---------------
Input::	▁언제든지 ▁신청해 주시기 ▁바랍니다 . 
Translation:	▁increasing ▁anytime . 
Target:	▁Please ▁register ▁anytime . 
---------------
Input::	▁피칭 ▁실력이 <unk> 에요 . 
Translation:	▁breaking ▁out ▁is ▁good . 
Target:	▁My ▁pitching ▁suck s . 
---------------
Input::	▁여기에 ▁드라이 기가 ▁있나요 ? 
Translation:	▁dry ▁hair ▁here ? 
Target:	▁Is ▁the ▁hairdryer ▁around ▁here ? 
---------------
Input::	▁팬들이 ▁엄청 ▁놀랐 겠다 . 
Translation:	be li vi d . ▁I ' m ▁surprised . 
Target:	▁Their ▁fans ▁must ▁be ▁shocked . 
---------------
Input::	▁저에게 ▁샌드위치 ▁좀 ▁주세요 . 
Translation:	▁Don ' t ▁give ▁me ▁a ▁sandwich . 
Target:	▁Please ▁give ▁me ▁some ▁sandwiches . 
---------------
Input::	▁스타벅스 ▁근처에 ▁식당이 ▁있어요 . 
Translation:	▁Starbucks ▁is ▁near ▁Starbucks . 
Target:	▁There ▁is ▁a ▁restaurant ▁near ▁Starbucks . 
---------------
Input::	▁지금 ▁생각나 려고 ▁합니다 . 
Translation:	▁I ' m ▁about ▁to ▁think .

In [40]:
get_number_of_params(model)

NameError: name 'get_number_of_params' is not defined

In [39]:
n_params/10**6

102.540544

120